# ***Basics Implementation of the Long-Term-Memory in ChatBot***

In [1]:
import uuid
from langgraph.store.memory import InMemoryStore

## Define the in memory store using build in class `InMemoryStore`

In [2]:
in_memory_store = InMemoryStore()

When storing objects (e.g., memories) in the [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore), we provide:

- The `namespace` for the object, a tuple (similar to directories)
- the object `key` (similar to filenames)
- the object `value` (similar to file contents)

We use the [put](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore.put) method to save an object to the store by `namespace` and `key`.

![langgraph_store.png](inmemory.png)

In [5]:
## Namespace for the memory
user_id = "1"
namespace_for_memory = (user_id, "memory")

# save a memory to namespace as key and values
key = str(uuid.uuid4())

# this is the memory context like value of the memory
value = {
    'food_preference': "I like beef"
}

## now save this info into the memory
in_memory_store.put(namespace=namespace_for_memory, key=key, value=value)

#### we use `search` to retrieve the object from the store by namespace
- This is return a list

In [ ]:
# search
memories = in_memory_store.search(namespace_for_memory)
type(memories)

list

In [ ]:
## Metadata
memories[0].dict()

{'namespace': ['1', 'memory'],
 'key': '222e9bfc-0d8b-4a98-a3f6-5efea24c04c0',
 'value': {'food_preference': 'I like beef'},
 'created_at': '2025-05-12T04:41:20.025246+00:00',
 'updated_at': '2025-05-12T04:41:20.025246+00:00',
 'score': None}

In [11]:
print(memories[0].key, memories[0].value)

222e9bfc-0d8b-4a98-a3f6-5efea24c04c0 {'food_preference': 'I like beef'}


We can also use [get](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore.get) to retrieve an object by `namespace` and `key`.

In [14]:
## get the memory by namespace and key
memory = in_memory_store.get(namespace_for_memory, key)
memory.dict()

{'namespace': ['1', 'memory'],
 'key': '222e9bfc-0d8b-4a98-a3f6-5efea24c04c0',
 'value': {'food_preference': 'I like beef'},
 'created_at': '2025-05-12T04:41:20.025246+00:00',
 'updated_at': '2025-05-12T04:41:20.025246+00:00'}

# ***Time to Build Chatbot with long-term-memory***